In [1]:
# import os
# os.environ['KERAS_BACKEND'] = 'theano'
# import keras 

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
from torchvision import transforms,datasets
import matplotlib.pyplot as plt
import torch.optim as optim

In [3]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(64),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(64),
                                      transforms.CenterCrop(64),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder('/home/nilesh/Desktop/MY FILES/DEEP LEARNING COURSE/3. Convolutional_Neural_Networks/Convolutional_Neural_Networks/dataset/training_set', transform=train_transforms)
test_data = datasets.ImageFolder('/home/nilesh/Desktop/MY FILES/DEEP LEARNING COURSE/3. Convolutional_Neural_Networks/Convolutional_Neural_Networks/dataset/test_set', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=100, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [4]:
_ , (images , labels) = next(enumerate(trainloader))
labels.shape , images.shape

(torch.Size([100]), torch.Size([100, 3, 64, 64]))

In [5]:
class neural_net(nn.Module):
    def __init__(self):
        super().__init__()
        #sees image of 64*64*3
        self.conv1 = nn.Conv2d(3 , 16 ,3 , padding = 1)
        #sees tensor of 32*32*16
        self.conv2 = nn.Conv2d(16 , 32 ,3 , padding = 1)
        #sees tensor of 16*16*32
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(16*16*32 , 500)
        #output layer
        self.fc2 = nn.Linear(500 , 2)
        self.dropout = nn.Dropout(0.25)
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1 , 16*16*32)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc2(x),dim=1)
        return x

In [6]:
model = neural_net()
model

neural_net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8192, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=2, bias=True)
  (dropout): Dropout(p=0.25)
)

In [7]:
model(images).shape

torch.Size([100, 2])

In [8]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters() , lr = 0.1)

In [9]:
# for _,(data,target) in enumerate(trainloader):
#     print(data)

In [10]:
for epoch in range(5):
    tot_loss=0
    for _,(data,target) in enumerate(trainloader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output , target)
        loss.backward()
        optimizer.step()
        tot_loss += loss.item()
    print('epoch' , epoch + 1, ': ' ,tot_loss/len(trainloader))

epoch 1 :  1232.6606129616498
epoch 2 :  0.6949506983160972
epoch 3 :  0.6941649422049523
epoch 4 :  0.6955553404986858
epoch 5 :  0.693646003305912


In [11]:
# data = iter(testloader)
# image , label = next(data)

In [12]:
_ , (test_images , test_labels) = next(enumerate(testloader))

In [13]:
pred = model(test_images)

In [15]:
pred_array = pred.detach().numpy()

In [18]:
ypred = [None for _ in range(len(pred_array))]
for i in range(len(pred_array)):
    maxm = -10000
    if maxm < pred_array[i][0] :
        maxm = pred_array[i][0]
        ypred[i] = 0
    if maxm < pred_array[i][1]:
        maxm = pred_array[i][0]
        ypred[i] = 1
len(ypred)

64

In [19]:
type(ypred)

list

In [21]:
print('accurcy:' , (ypred == test_labels.detach().numpy()).sum()/len(ypred))

accurcy: 1.0
